# Install & Uninstall


> Executar uma unica vez



In [ ]:
!pip install Cython==0.29.36
!pip uninstall scipy -y
!pip install scipy==1.9
!pip uninstall pyparsing -y
!pip install pyparsing==2.4
!pip uninstall scikit_learn -y

In [ ]:
!pip uninstall imbalanced-learn -y
!pip uninstall mlxtend -y
!pip uninstall yellowbrick -y

In [ ]:
!pip install scikit-learn==0.24.2 --no-build-isolation

In [ ]:
!pip install auto-sklearn

# K-Nearest Neighbors (K-NN)

## Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import autosklearn
from autosklearn.metrics import (accuracy,
                                 f1,
                                 roc_auc,
                                 precision,
                                 average_precision,
                                 recall,
                                 log_loss)

## Importing the dataset

In [ ]:
dataset = pd.read_csv('HIV.csv', sep=';')
X = dataset.iloc[:, 5:-1].values
y = dataset.iloc[:, -1].values

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.fit_transform(X_test)

## Training the K-NN model on the Training set

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 7, metric = 'minkowski', p = 2)
classifier.fit(X_train_pca, y_train)

KNeighborsClassifier(n_neighbors=7)

## Predicting the Test set results

In [ ]:
y_pred = classifier.predict(X_test_pca)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [1 1]
 ...
 [1 3]
 [1 1]
 [1 1]]


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Making the Confusion Matrix & Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[503  49   0   0   0]
 [ 64   0   0   0   0]
 [ 25   0   0   0   2]
 [ 12   0   0   0   0]
 [  8   1   0   0   0]]


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,
                           classifier.predict(X_test_pca)))

              precision    recall  f1-score   support

           1       0.82      0.91      0.86       552
           2       0.00      0.00      0.00        64
           3       0.00      0.00      0.00        27
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00         9

    accuracy                           0.76       664
   macro avg       0.16      0.18      0.17       664
weighted avg       0.68      0.76      0.72       664



/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined an

# Auto-Sklearn

## Training the Auto-Sklearn model on the Training set

In [ ]:
import autosklearn.classification as classifier

classifier = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
)

classifier.fit(X_train_pca, y_train)

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      per_run_time_limit=180, time_left_for_this_task=1800)

## Leaderboard & Hyperparameters

In [ ]:
print(classifier.leaderboard())

          rank  ensemble_weight                 type      cost   duration
model_id                                                                 
18           1             0.14        random_forest  0.100304   3.018579
247          2             0.12        random_forest  0.104863   2.550339
291          3             0.02        random_forest  0.107903   3.236886
210          4             0.04        random_forest  0.109422   3.272823
71           5             0.02        random_forest  0.110942   2.682914
206          7             0.02        random_forest  0.110942   3.487064
254          6             0.04        random_forest  0.110942   2.338216
82           8             0.06        random_forest  0.112462   2.513035
177          9             0.06        random_forest  0.113982   2.547721
186         10             0.02        random_forest  0.113982   2.596955
119         11             0.14        random_forest  0.115502   2.335843
20          13             0.04       

In [ ]:
print(classifier.show_models())

{18: {'model_id': 18, 'rank': 1, 'cost': 0.10030395136778114, 'ensemble_weight': 0.14, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fa5ed6bfc10>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fa5e99a7040>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fa5e99a75e0>, 'sklearn_classifier': RandomForestClassifier(criterion='entropy', max_features=1, min_samples_split=5,
                       n_estimators=512, n_jobs=1, random_state=1,
                       warm_start=True)}, 20: {'model_id': 20, 'rank': 2, 'cost': 0.11702127659574468, 'ensemble_weight': 0.04, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fa5ee37a470>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.compo

## Predicting the Test set results

In [ ]:
y_pred = classifier.predict(X_test_pca)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [1 1]
 ...
 [1 3]
 [1 1]
 [1 1]]


## Making the Confusion Matrix & Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[515  37   0   0   0]
 [ 64   0   0   0   0]
 [ 25   0   2   0   0]
 [ 12   0   0   0   0]
 [  8   1   0   0   0]]


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,
                           classifier.predict(X_test_pca)))

              precision    recall  f1-score   support

           1       0.83      0.93      0.88       552
           2       0.00      0.00      0.00        64
           3       1.00      0.07      0.14        27
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00         9

    accuracy                           0.78       664
   macro avg       0.37      0.20      0.20       664
weighted avg       0.73      0.78      0.73       664



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
